In [1]:
import re
import random
import numpy as np
import pandas as pd

In [2]:
pat = '[a-zA-Z0-9\(+\*]+|\)'
RIGHTB = ')'
LEFTB = '('
STAR = '*'
EMPTY = ''
SPACE = ' '

In [3]:
notation = "sC mC fC( hC( + sB mB fB( hB( + sB* ))))"

In [4]:
def complement(s):
    if STAR in s: # If the domain contains an asterisk, remove it.
        s = s.replace(STAR, EMPTY)
    else: # If not, add an asterisk to the end.
        s = s + STAR
    return s
    
def brace_seen_transform(left_brace_str, right_brace_str):
    right_brace_str = left_brace_str.replace(LEFTB, EMPTY) # Remove the left brace from the string with left brace.
    right_brace_str = complement(right_brace_str)
    right_brace_str += RIGHTB # Finally add a left brace.
    return (left_brace_str, right_brace_str)

def shadow(s):
    sequence = Sequence(s)
    domains = []
    for d in sequence.domains:
        if LEFTB in d.seq:
            d.seq = d.seq.replace(LEFTB, RIGHTB)
            domains.append(d)
            continue
        elif RIGHTB in d.seq:
            d.seq = d.seq.replace(RIGHTB, LEFTB)
            domains.append(d) 
            continue
        else:
            domains.append(d)
    domains.reverse()
    domain_strs = [d.seq if RIGHTB not in d.seq else RIGHTB for d in domains]
    return SPACE.join(domain_strs)

brace_seen_transform('sC*(', ')')

('sC*(', 'sC)')

In [5]:
class Domain:
    def __init__(self, s):
        self.seq = s
    
    def _reverse(self):
        return Domain(self.seq + 'R')
    
    def _reverse_compl(self):
        return Domain(self.seq + 'R' + '*')

class Sequence:
    
    def __init__(self, s: str):
        list_of_strs = re.findall(pat, s)
        self.domains = [
            Domain(s) for s in list_of_strs
        ]
        brackets_stack = []
        for i in range(len(self.domains)):
            if LEFTB in self.domains[i].seq: # The domain contains a left bracket.
                brackets_stack.append((self.domains[i].seq, i)) # Add the domain to the stack.
                
            if RIGHTB in self.domains[i].seq: # If the domain contains a right bracket.
                left_brace_str, leftindex = brackets_stack.pop()
                right_brace_str = self.domains[i].seq
                
                left_brace_str, right_brace_str = brace_seen_transform(left_brace_str, right_brace_str)
                
                self.domains[leftindex] = Domain(left_brace_str)
                self.domains[i] = Domain(right_brace_str)
    
    
        
    
    def __str__(self):
        domains = [d.seq if RIGHTB not in d.seq else RIGHTB for d in self.domains]
        return SPACE.join(domains)



In [8]:
notation = "sC mC fC( hCk( + sC mC fC( hCj( + fB* ) ) ) )"

In [9]:

shadow_seq = shadow(notation)

subs = list(set(re.findall('[a-zA-Z]+', shadow_seq)))
for sub in subs:
    shadow_seq = shadow_seq.replace(sub, sub+'R')


print(shadow_seq)

fCR*( hCkR*( fCR*( hCjR*( fBR* + ) ) mCR sCR + ) ) mCR sCR
